In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os, platform
system = platform.platform()
system = system[0:system.index('-')]
print("Your operating system:", system)

if system == "Linux":
    dataDir = '/kaggle/input/isic-2024-challenge/'
    print(os.listdir(dataDir))
    model_path = "/kaggle/input/resnet18/pytorch/v1-2epochs/1/best_model.pth"
elif system == "macOS":
    dataDir = 'Data/'
    print(os.listdir(dataDir))
    model_path = "best_model.pth"
else:
    raise Exception("Please manually set the `dataDir` variable to the directory where the data files are located")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Your operating system: macOS
['.DS_Store', 'train-image', 'train-metadata.csv', 'submission.csv', 'test-image.hdf5', 'train-image.hdf5', 'test-metadata.csv', 'split.npz', 'sample_submission.csv']


In [2]:
import torch

if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
    
device

device(type='mps')

In [3]:
from torchvision.models import resnet18, ResNet18_Weights
import torch.nn as nn

from PIL import Image
import h5py
import io

def load_batch(hdf5, transforms, keys, device):
    with h5py.File(hdf5, "r") as f:
        imgs = [transforms(Image.open(io.BytesIO(f.get(key)[()]))) for key in keys]
    X = torch.stack(imgs).to(device)
    return X

hdf5 = dataDir + "test-image.hdf5"
transforms = ResNet18_Weights.IMAGENET1K_V1.transforms()

model = resnet18()
model.fc = nn.Linear(model.fc.in_features, 1)
model.load_state_dict(torch.load(model_path, map_location=device))
model = model.to(device)
model.eval();

sigmoid = nn.Sigmoid().to(device)
sigmoid.eval();

In [4]:
from tqdm.notebook import tqdm

batch_size = 1024
submission = pd.read_csv(dataDir + "sample_submission.csv")
ids = list(submission['isic_id'])
tmp = submission.to_numpy()[:, -1]
tmp = tmp.reshape([len(tmp), 1])

for i in tqdm(range(0, len(ids), batch_size)):
    with torch.no_grad():
        keys = ids[i:i+batch_size]
        X = load_batch(hdf5, transforms, ids[i:i+batch_size], device)
        outputs = sigmoid(model(X))
        tmp[i:i+batch_size] = outputs.cpu().numpy()
    
    submission["target"] = tmp.reshape(len(tmp))

if system == "Linux":
    dataDir = "/kaggle/working/"

submission.to_csv(dataDir + "submission.csv", index=None)
submission

  0%|          | 0/1 [00:00<?, ?it/s]

,isic_id,target
0,ISIC_0015657,0.110094
1,ISIC_0015729,0.009171
2,ISIC_0015740,0.114385
